LSTM MODEL

In [39]:
#Import all the libraries
from keras.models import load_model
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go
import plotly.express as px
from scipy.signal import medfilt, butter, filtfilt
import pywt
from sklearn.model_selection import train_test_split
import scipy.signal
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
model = load_model('model.h5')

In [40]:
from numpy import shape


def featureselection(X_train):
    X_train=np.array(X_train)
    features = []
    print(X_train.shape)
    # Extracting features for each sample
    for i in range(1):
        #Finding the R-peaks
        print(X_train[i])
        r_peaks = scipy.signal.find_peaks(X_train[i])[0]
        #Initialize lists to hold R-peak and T-peak amplitudes
        r_amplitudes = []
        t_amplitudes = []
        # Iterate through R-peak locations to find corresponding T-peak amplitudes
        for r_peak in r_peaks:
        # Find the index of the T-peak (minimum value) in the interval from R-peak to R-peak + 200 samples
            t_peak = np.argmin(X_train[i][r_peak:r_peak+200]) + r_peak
            #Append the R-peak amplitude and T-peak amplitude to the lists
            r_amplitudes.append(X_train[i][r_peak])
            t_amplitudes.append(X_train[i][t_peak])
        # extracting singular value metrics from the r_amplitudes
        std_r_amp = np.std(r_amplitudes)
        mean_r_amp = np.mean(r_amplitudes)
        median_r_amp = np.median(r_amplitudes)
        sum_r_amp = np.sum(r_amplitudes)
        # extracting singular value metrics from the t_amplitudes
        std_t_amp = np.std(t_amplitudes)
        mean_t_amp = np.mean(t_amplitudes)
        median_t_amp = np.median(t_amplitudes)
        sum_t_amp = np.sum(t_amplitudes)
        # Find the time between consecutive R-peaks
        rr_intervals = np.diff(r_peaks)
        # Calculate the time duration of the data collection
        time_duration = (len(X_train[i]) - 1) / 1000 # assuming data is in ms
        # Calculate the sampling rate
        sampling_rate = len(X_train[i]) / time_duration
        # Calculate heart rate
        duration = len(X_train[i]) / sampling_rate
        heart_rate = (len(r_peaks) / duration) * 60
        # QRS duration
        qrs_duration = []
        for j in range(len(r_peaks)):
            qrs_duration.append(r_peaks[j]-r_peaks[j-1])
        # extracting singular value metrics from the qrs_durations
        std_qrs = np.std(qrs_duration)
        mean_qrs = np.mean(qrs_duration)
        median_qrs = np.median(qrs_duration)
        sum_qrs = np.sum(qrs_duration)
        # Extracting the singular value metrics from the RR-interval
        std_rr = np.std(rr_intervals)
        mean_rr = np.mean(rr_intervals)
        median_rr = np.median(rr_intervals)
        sum_rr = np.sum(rr_intervals)
        # Extracting the overall standard deviation
        std = np.std(X_train[i])
        # Extracting the overall mean
        mean = np.mean(X_train[i])
        # Appending the features to the list
        features.append([mean, std, std_qrs, mean_qrs,median_qrs, sum_qrs, std_r_amp, mean_r_amp, median_r_amp, sum_r_amp, std_t_amp, mean_t_amp, median_t_amp, sum_t_amp, sum_rr, std_rr, mean_rr,median_rr, heart_rate])
    # Converting the list to a numpy array
    features = np.array(features)
    num_features = features.shape[1]
    # Reshape the features data to be in the right shape for LSTM input
    features = features.reshape(features.shape[0], features.shape[1], 1)
    print(features)
    return features

In [41]:
def format_data(ecg_data):
    ecg_data=pd.DataFrame(ecg_data)
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(-1, 1))
    ecg_data = scaler.fit_transform(ecg_data)
    from scipy.signal import medfilt, butter, filtfilt
    import pywt
    import numpy as np
    #filtering the raw signals
    # Median filtering
    ecg_medfilt = medfilt(ecg_data, kernel_size=3)
    # Low-pass filtering
    lowcut = 0.05
    highcut = 20.0
    nyquist = 0.5 * 360.0
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(4, [low, high], btype='band')
    ecg_lowpass = filtfilt(b, a, ecg_data)
    # Wavelet filtering
    coeffs = pywt.wavedec(ecg_data, 'db4', level=1)
    threshold = np.std(coeffs[-1]) * np.sqrt(2*np.log(len(ecg_data)))
    coeffs[1:] = (pywt.threshold(i, value=threshold, mode='soft') for i in coeffs[1:])
    ecg_wavelet = pywt.waverec(coeffs, 'db4')
    return ecg_wavelet

# This is the final function that you have to use for prediction. But the above 2 functions are needed to be called within this function. The input to this function is a list of ECG values similar to 1 row of the dataset.(140 values). Normalization and feature selection would be done automatically.


In [44]:
def model_predict(l):
    test=format_data(l)
    test=featureselection(l)
    pred=model.predict(test)
    if pred>=0.5:
        return 0
    else:
        return 1
a = []
out = np.array(a)
print(model_predict([out]))

(1, 140)
[-0.48104583 -0.89507291 -1.6139316  -2.3150379  -2.8132684  -3.4483725
 -3.7958751  -3.724429   -3.233412   -2.5728547  -1.7754831  -1.3752221
 -1.2717754  -1.1769392  -0.69896056 -0.32767706 -0.34774622 -0.25987326
 -0.29708395 -0.26650075 -0.19947271 -0.23269771 -0.12767905 -0.1117111
 -0.21274644 -0.23851323 -0.29606666 -0.18276302 -0.23140758 -0.27732169
 -0.21509662 -0.15659865 -0.29736812 -0.34161522 -0.20570406 -0.17081856
 -0.23600654 -0.25926463 -0.23550374 -0.20075316 -0.09247985 -0.10157705
 -0.21069194 -0.23446761 -0.26356206 -0.23040141 -0.25739743 -0.33296364
 -0.30888389 -0.25655495 -0.20475829 -0.26725194 -0.29689288 -0.2736634
 -0.27730672 -0.2756215  -0.27679635 -0.34274949 -0.34614008 -0.32344645
 -0.29953083 -0.33101675 -0.34223774 -0.24952694 -0.31030925 -0.27540634
 -0.26717798 -0.31483612 -0.26695004 -0.21320208 -0.14146621 -0.28237629
 -0.11932984  0.06128687 -0.0161068   0.02818601 -0.00589579  0.05898221
  0.0302873   0.05980457  0.13481763  0.051913

c:\Users\samab\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\signal\_signaltools.py:1551: UserWarning: kernel_size exceeds volume extent: the volume will be zero-padded.
  warnings.warn('kernel_size exceeds volume extent: the volume will be '
